Pandas_Lecture_03.pdf

# Pandas 심화

## 1. 조건으로 검색하기
- numpy array와 마찬가지로 masking 연산이 가능하다

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.rand(5, 2), columns=["A", "B"])
df

,A,B
0,0.862572,0.941879
1,0.625943,0.284457
2,0.618768,0.678435
3,0.525505,0.443253
4,0.856647,0.388774


In [2]:
df['A'] < 0.5

0    False
1    False
2    False
3    False
4    False
Name: A, dtype: bool

- 조건에 맞는 열 추출이 가능하다

In [3]:
df = pd.DataFrame(np.random.rand(5, 2), columns=["A", "B"])
[(df["A"] < 0.5) & (df["B"] > 0.3)]

[0    False
 1     True
 2    False
 3    False
 4     True
 dtype: bool]

In [4]:
df.query("A < 0.5 and B > 0.3")

,A,B
1,0.239309,0.790133
4,0.495534,0.906543


## 2. 함수로 데이터 처리하기
- apply 사용

- #1

In [5]:
df = pd.DataFrame(np.arange(5), columns=["Num"])
df

,Num
0,0
1,1
2,2
3,3
4,4


In [6]:
def square(x):
    return x**2

df["Num"].apply(square)
df["Square"] = df.Num.apply(lambda x: x ** 2)
df

,Num,Square
0,0,0
1,1,1
2,2,4
3,3,9
4,4,16


- #2

In [7]:
df = pd.DataFrame(columns=["phone"])
df.loc[0] = "010-1234-1235"
df.loc[1] = "공일공-일이삼사-1235"
df.loc[2] = "010.1234.일이삼오"
df.loc[3] = "공1공-1234.1이3오"
df["preprocess_phone"] = ''
df

,phone,preprocess_phone
0,010-1234-1235,
1,공일공-일이삼사-1235,
2,010.1234.일이삼오,
3,공1공-1234.1이3오,


In [8]:
def get_preprocess_phone(phone):
    mapping_dict = { # 사전으로 맵핑해서 자동화시킴
        "공": "0",
        "일": "1",
        "이": "2",
        "삼": "3",
        "사": "4",
        "오": "5",
        "-": "",
        ".": "",
    }

    for key, value in mapping_dict.items(): # 입력값 두개를 받는 item() 함수
        phone = phone.replace(key, value)
        return phone

df["preprocess_phone"] = df["phone"].apply(get_preprocess_phone)
df

,phone,preprocess_phone
0,010-1234-1235,010-1234-1235
1,공일공-일이삼사-1235,0일0-일이삼사-1235
2,010.1234.일이삼오,010.1234.일이삼오
3,공1공-1234.1이3오,010-1234.1이3오


- replace 대체

In [9]:
sex_dict = {
    0 : 'Male', 
    1 : 'Male', 
    2 : 'Female', 
    3 : 'Female',
    4 : 'Male'
}

sex_Series = pd.Series(sex_dict)

df1 = pd.DataFrame({'sex': sex_Series})

df1

,sex
0,Male
1,Male
2,Female
3,Female
4,Male


In [10]:
df1.sex

0      Male
1      Male
2    Female
3    Female
4      Male
Name: sex, dtype: object

In [11]:
df1.sex.replace({"Male": 0, "Female": 1})
df1

,sex
0,Male
1,Male
2,Female
3,Female
4,Male


In [12]:
df1.sex.replace({"Male": 0, "Female": 1}, inplace=True)
df1

,sex
0,0
1,0
2,1
3,1
4,0


## 3. 그룹으로 묶기
- 조건부로 집계하고 싶은 경우

### groupby

In [13]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(6)})
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [14]:
df.groupby('key')

In [15]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [16]:
df.groupby(['key','data']).sum()

Empty DataFrame
Columns: []
Index: [(A, 0), (A, 3), (B, 1), (B, 4), (C, 2), (C, 5)]

### aggregate: 반드시 groupby 통해서 집계를 한번에!

In [17]:
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [18]:
df.columns

Index(['key', 'data'], dtype='object')

In [19]:
df.rename(columns = {'data' : 'data1'}, inplace = True)
df['data2'] = [4,4,6,0,6,1]
df

,key,data1,data2
0,A,0,4
1,B,1,4
2,C,2,6
3,A,3,0
4,B,4,6
5,C,5,1


In [20]:
df.groupby('key').aggregate(['min', np.median, max])
# key를 기준으로 묶어서 최솟값, 중간값, 최대값 구하기

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     0    2.0   4
B       1    2.5   4     4    5.0   6
C       2    3.5   5     1    3.5   6

In [21]:
df.groupby('key').aggregate({'data1': 'min', 'data2': np.sum})

,data1,data2
key,,
A,0,4
B,1,10
C,2,7


### fillter: 
- groupby를 통해 그룹 속성을 기준으로 데이터 필터링

In [22]:
df

,key,data1,data2
0,A,0,4
1,B,1,4
2,C,2,6
3,A,3,0
4,B,4,6
5,C,5,1


In [23]:
def filter_by_mean(x):
    return x['data2'].mean() > 3

In [24]:
df.groupby('key')
# <pandas.core.groupby.groupby.DataFrameGroupBy object at 0x10e3588>

In [25]:
df.groupby('key').mean() # 평균값

,data1,data2
key,,
A,1.5,2.0
B,2.5,5.0
C,3.5,3.5


In [26]:
df.groupby('key').filter(filter_by_mean)

,key,data1,data2
1,B,1,4
2,C,2,6
4,B,4,6
5,C,5,1


### apply

In [27]:
df

,key,data1,data2
0,A,0,4
1,B,1,4
2,C,2,6
3,A,3,0
4,B,4,6
5,C,5,1


In [28]:
df.groupby('key').apply(lambda x: x.max() - x.min()) # x.max()~x.min() 범위

,data1,data2
key,,
A,3,4
B,3,2
C,3,5


### get_group
- groupby로 묶인 데이터에서 key값으로 데이터를 가져올 수 있다

In [29]:
# df = pd.read_csv("./univ.csv")
# df.head()
# df.groupby("시도").get_group("충남")
# len(df.groupby("시도").get_group("충남"))

## 4. MultiIndex & pivot_table

### 멀티인덱스 
- 인덱스를 계층적으로 만들 수 있다.

In [30]:
df = pd.DataFrame(
np.random.randn(4, 2), 
index=[['A', 'A', 'B', 'B'], [1, 2, 1, 2]],
columns=['data1', 'data2']
)
df

data1     data2
A 1 -1.072529 -1.985974
  2 -0.716800  0.875379
B 1  0.057293 -1.319529
  2  0.331123  0.268629

- 열 인덱스도 계층적으로 만들 수 있다

In [31]:
df = pd.DataFrame(np.random.randn(4, 4),
                  columns=[["A", "A", "B", "B"], ["1", "2", "1", "2"]])
df

A                   B          
          1         2         1         2
0 -2.253816 -0.982722 -0.870866  0.828567
1 -0.383495 -0.990350  0.171906 -1.392499
2 -1.506086  0.109788 -0.959946 -0.836192
3  0.607993  1.249663 -0.338445 -0.630462

- 다중 인덱스 컬럼의 경우 인덱싱은 계층적하며
- loc, iloc를 사용한다.

In [32]:
df['A']

,1,2
0,-2.253816,-0.982722
1,-0.383495,-0.990350
2,-1.506086,0.109788
3,0.607993,1.249663


In [33]:
df['A']['1']

0   -2.253816
1   -0.383495
2   -1.506086
3    0.607993
Name: 1, dtype: float64

### 피벗테이블 pivot_table
- 데이터에서 필요한 자료만 뽑아서 새롭게 요약, 분석한다.
- 엑셀과 비슷
- Index : 행 인덱스로 들어갈 key
- Column : 열 인덱스로 라벨링될 값
- Value : 분석할 데이터

In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/TeamLab/machine_learning_from_scratch_with_python/master/code/ch12/titanic/train.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
df.pivot_table(
    index='Sex', columns='Pclass', values='Survived',
    aggfunc=np.mean
)

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [36]:
# df.pivot_table(index="월별", columns='내역', values=["수입", '지출'])